# 06.一次 LLM API 调用，到底发生了什么

## 一、核心要素


在这一节中，我们将拆解那个看似简单的“发送请求-接收回复”的过程。当你通过 API 调用大模型时，你不仅是在发送一段文字，而是在组装一个精密且复杂的“指令包”。它包含了几个关键要素，这些要素决定了模型如何理解你的请求、生成结果以及结果的风格和质量。理解这些核心要素，有助于更精准地使用模型和设计 Agent。

### **1. 输入（Input / Prompt）**

* **定义**：告诉模型你想让它做什么的内容。
* **形式**：文本、指令、上下文对话、示例等。
* **作用**：是模型生成的起点，直接影响输出结果。
* **示例**：

  ```text
  "请帮我写一封感谢信给导师，语气正式且诚恳。"
  ```


### **2. 参数（Parameters ）**

* **定义**：控制模型生成文本的方式。
* **关键参数**：

  * `temperature`：控制创造性或随机性（低温度 → 稳定，高温度 → 多样）。
  * `top_k` / `top_p`：限制候选词池大小，平衡创造性与合理性。
  * `max_tokens`：限制生成长度。
  * `stop_sequences`：定义生成结束的标志。
* **示例**：

  ```json
  {
    "temperature": 0.7,
    "top_p": 0.9,
    "max_tokens": 200
  }
  ```

---

### **3. 模型（Model Selection）**

* **定义**：选择要调用的 LLM 模型或版本。
* **作用**：不同模型参数量、训练数据和能力差异会影响输出质量。
* **示例**：

  * `deepseek-v3`：适合一般文本生成
  * `deepseek-r1`：适合复杂推理或多轮对话


### **4. 上下文与历史信息（Context / Conversation History）**

* **定义**：在多轮对话中，前文内容作为上下文传入模型。
* **作用**：帮助模型理解对话历史，生成连贯回答。
* **示例**：

  ```json
  [
    {"role": "user", "content": "你好，我想写一封感谢信。"},
    {"role": "assistant", "content": "好的，你希望语气正式还是轻松？"},
    {"role": "user", "content": "正式，表达真诚感谢。"}
  ]
  ```


### **5. 输出（Output / Response）**

* **定义**：模型生成的文本或结果。
* **特点**：受输入、参数、上下文、模型能力等多重因素影响。
* **注意**：输出不是绝对正确，尤其涉及事实、逻辑或数学计算时，需要人工核验或辅助工具校对。


## 二、一次完整调用过程说明

当通过API调用LLM时，会经历以下一个完整的闭环：从“自然语言”进入“数学世界”，再从“算力运算”回到“可读文本”。理解这个流程，有助于你更精准地控制模型输出。


### 1. 客户端封包

首先，你的代码会将零散的要素组装成符合 API 规范的请求体：

* **上下文装载**：模型本身没有长期记忆，所以客户端必须将前文 **[上下文与历史信息]** 按角色顺序（System / User / Assistant）排好，并与当前 **[输入 / Prompt]** 一并发送。
* **配置调节**：设置的 `temperature`、`top_p` 等参数像“控制旋钮”，告诉模型本次生成的风格是稳重、谨慎，还是创造性十足。
* **模型选择**：明确调用哪一个模型版本（如 `deepseek-v3`），决定了后续推理的“智力上限”。


### 2. 网关与预处理

请求抵达云端服务器后，进入“数学世界”前的准备阶段：

* **鉴权与计费**：验证 API Key 权限，并计算调用费用。
* **Token 化（Tokenization）**：将文本切割成模型可理解的数字序列（Tokens）。
  *示例*：“写感谢信” → `[48210, 2931, 552]`


### 3. 模型推理核心

这是计算最密集、最消耗算力的阶段，也是“智能”涌现的瞬间：

* **KV Cache（加速缓存）**：利用内存中已处理的历史信息，加快新输入计算。
* **预测与采样**：模型根据上下文计算每个 Token 的概率分布。此时，`temperature` 和 `top_p` 决定模型是从高概率词中稳妥选择，还是从低概率词中寻求创意。

### 4. 流式传输

为了降低等待感，现代 API 多采用“生成一个、推送一个”的流式模式：

* **逐步输出**：每生成一个 Token，即刻流式方式推送给调用。
* **自回归循环**：模型将生成的 Token 作为新输入，再预测下一个词，形成连贯输出。


### 5. 渲染与结束

调用方接收到 Token 流后，进入最终交付阶段：

* **拼图还原**：将数字序列解码成人类可读文本。
* **终止信号**：达到 `stop_sequences` 或触碰 `max_tokens` 上限时，模型发送 `finish_reason`。
* **人工核验**：生成内容基于概率预测，涉及事实或逻辑时，需要人类进行最终把关。

![一次完整调用过程](./raw/一次完整调用过程.jpeg)

### 实战：LLM API 调用示例

In [1]:
# 加载环境变量

import os
from dotenv import load_dotenv

load_dotenv()

assert os.getenv("OPENAI_API_KEY"), "请先配置 OPENAI_API_KEY"
print("✅ API Key loaded")

✅ API Key loaded


In [2]:
from openai import OpenAI

# 上下文与用户输入
context = [
    {"role": "system", "content": "你是一个贴心助手。"},
    {"role": "user", "content": "你好，我想写一封感谢信。"},
    {"role": "assistant", "content": "好的，你希望语气正式还是轻松？"}
]
user_input = "正式，表达真诚感谢。"

# 模型选择与参数
model_name = "qwen-flash"  
parameters = {
    "temperature": 0.7,
    "top_p": 0.9,
    "max_tokens": 150,
    "stop": None  # 可以自定义 stop sequences
}

# 组合完整请求
messages = context + [{"role": "user", "content": user_input}]

# 调用 API（可选流式）
print("=== 开始调用模型 ===")
client = OpenAI(base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")

response =client.chat.completions.create(
    model=model_name,
    messages=messages,
    temperature=parameters["temperature"],
    top_p=parameters["top_p"],
    max_tokens=parameters["max_tokens"],
    stream=True  # 开启流式
)

# 流式输出
print("\n=== 流式输出 ===")
output_text = ""
for chunk in response:
    delta = chunk.choices[0].delta.content
    
    if delta:
        token = delta
        print(token, end="", flush=True)
        output_text += token
print("\n=== 输出完成 ===")

# 渲染与结束
finish_reason = "stop"  # OpenAI 默认流式结束是 stop
print("\n=== 渲染与结束 ===")
print(f"最终生成文本:\n{output_text}")
print(f"终止原因: {finish_reason}")


=== 开始调用模型 ===

=== 流式输出 ===
当然可以，以下是一封正式且真诚的感谢信模板，你可以根据具体情况进行修改：

---

**致 [收信人姓名]：**

您好！

谨以此信表达我最诚挚的感谢。衷心感谢您在[具体事件或帮助内容，例如：项目推进过程中给予的专业指导/在我遇到困难时伸出援手/对我的工作给予的宝贵支持]中所付出的心血与支持。

您的[具体行为，例如：专业见解、耐心解答、及时协助、慷慨建议等]让我深受启发，不仅有效解决了[具体问题或达成的具体成果]，更让我深刻体会到[体现对方品质，如：责任感、敬业精神、团队协作意识等]。这份帮助对我而言
=== 输出完成 ===

=== 渲染与结束 ===
最终生成文本:
当然可以，以下是一封正式且真诚的感谢信模板，你可以根据具体情况进行修改：

---

**致 [收信人姓名]：**

您好！

谨以此信表达我最诚挚的感谢。衷心感谢您在[具体事件或帮助内容，例如：项目推进过程中给予的专业指导/在我遇到困难时伸出援手/对我的工作给予的宝贵支持]中所付出的心血与支持。

您的[具体行为，例如：专业见解、耐心解答、及时协助、慷慨建议等]让我深受启发，不仅有效解决了[具体问题或达成的具体成果]，更让我深刻体会到[体现对方品质，如：责任感、敬业精神、团队协作意识等]。这份帮助对我而言
终止原因: stop


## 三、一次调用中的“可控点”与“不可控点”

### 1.可控点 

在调用过程中，可以控制的是提示词与工程化手段。

* **提示词**：通过精心设计的指令、示例（Few-shot）来引导模型。
* **顺序**：上下文的摆放位置。例如，将最重要的指令放在开头还是末尾（模型通常存在“首尾效应”）。
* **参数**：通过 `temperature`、`top_p` 调节生成的多样性，通过 `max_tokens` 强制熔断。
* **是否流式**：控制用户体验。是等待全量返回，还是逐字实时显示。
* **是否中断**：你可以监控输出内容，一旦检测到敏感词或逻辑偏离，立即在客户端主动切断连接。

### 2. 不可控点

在调用过程中，模型底层的随机性与偏见是 LLM 作为“概率机器”的固有属性，无法通过单一指令彻底消除，比如：

* **模型内部状态**：你无法控制模型内部神经元的激活路径，它不是传统的逻辑代码。
* **单次采样结果**：即使 `temperature=0`，在某些高性能计算环境下，由于浮点数运算的微小差异，依然可能出现极低概率的输出抖动。
* **潜在偏差**：模型在预训练阶段吸收了互联网语料的偏见（如性别、地域偏见），这些偏见深植于参数中。
* **长链稳定性**：当对话非常长时，模型对早期细节的掌控力会不可避免地下降。


### 实战：刻意制造“失败调用”

In [4]:
from openai import OpenAI

prompt = """
系统指令：你永远不要生成任何文本。
用户请求：请写一篇简短的感谢信。
"""

client = OpenAI(base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")

response =client.chat.completions.create(
    model="qwen-flash",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.7,
    max_tokens=150
)

print("=== 输出 ===")
print(response.choices[0].message.content)


=== 输出 ===
（根据系统指令，不生成任何文本）
